# Introduction

Running a simmulation with SpatialPy requires only 2 components: a model (data), and a solver (algorithm).

## Basics

### Setup the Environment

'os' and 'sys' allow us to modify the directory Python searches for source code. If you wish to use an installed GillesPy2 package, you do not need to run this cell. If this notebook is being run from within the GillesPy2 source code directory we need to let Python know that we want to use it, not the GillesPy2 package.

In [ ]:
import os
import sys
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), '../')))

Numpy is used to set the timespan of the Model.

In [ ]:
import numpy

Import the types that'll be needed to define your Model.

In [ ]:
from spatialpy import (
    Model,
    Domain,
    Species,
    ScatterInitialCondition,
    Parameters,
    Reactions
)

## Creating a SpatialPy Model

A SpatialPy model must include a domain.

Your model is declared and configured as a Python class. As such, the name between `class` and `(Model):` can be of your choosing.
For this example we'll be modeling Spatial Birth and Death, so lets set the name accordingly.

In [ ]:
class SpatialBirthDeath(Model):
    # Define type IDs as constants of the Model
    HABITAT = "Habitat"

    def __init__(self):
        
        # Intialize the Model with a name of your choosing.
        Model.__init__(self, model_name='Spatial Birth-Death')
        
        """
        A domain defines points and attributes of a regional space for simulation.
        
        - xlim: highest and lowest coordinate in the x dimension.
        - ylim: highest and lowest coordinate in the y dimension.
        - zlim: highest and lowest coordinate in the z dimension.
        - nx: number of particle spacing in the x dimension.
        - ny: number of particle spacing in the y dimension.
        - nz: number of particle spacing in the z dimension.
        - type_id: default type ID of particles to be created. Defaults to 1.
        - mass: default mass of particles to be created. Defaults to 1.0.
        - nu: default viscosity of particles to be created. Defaults to 1.0.
        - c: default artificial speed of sound of particles to be created. Defaults to 0.0.
        - rho: default density of particles to be created.
        - fixed: spatially fixed flag of particles to be created. Defaults to false.
        - rho: Background density for the system
        - c0: Speed of sound for the system
        - P0: Background pressure for the system
        - gravity: Acceleration of gravity for the system.
        """
        domain = Domain.create_3D_domain(
            xlim=(0, 1), ylim=(0, 1), zlim=(0, 0.1), nx=10, ny=10, nz=1, type_id=self.HABITAT, fixed=True
        )
        
        # Add the Domain to the Model
        self.add_domain(domain)
        
        """
        Species can be anything that participates in or is produced by a reaction channel.

        - name: A user defined name for the species.
        - diffusion_coefficient: Non-constant coefficient of diffusion for the Species.
        - restrict_to: Restricts the movement of Species to  a type or list of types within the Domain.
        """
        Rabbits = Species(name='Rabbits', diffusion_coefficient=0.1)
        
        # Add the Species to the Model.
        self.add_species(Rabbits)
        
        """
        Scatter the initial condition for a species randomly over the list of types.
        If the list of types is None scatter over all types.
        
        - species: The species to set the initial condition.
        - count: The initial condition for the target species.
        - types: Type IDs of the particles to place the initial condition.
        """
        init_rabbit_pop = ScatterInitialCondition(species=rabbits, count=100)
        
        # Add Initial Conditions to the Model
        self.add_initial_condition(init_rabbit_pop)
        
        """
        Parameters are constant values relevant to the system, such as reaction kinetic rates.

        - name: A user defined name for reference.
        - expression: Some constant value.
        """
        kb = Parameter(name='k_birth', expression=10)
        kd = Parameter(name='k_death', expression=0.1)
        
        # Add the Parameters to the Model.
        self.add_parameter([kb, kd])
        
        """
        Reactions are the reaction channels which cause the system to change over time.

        - name: A user defined name for the reaction.
        - reactants: A dictionary with participant reactants as keys, and consumed per reaction as value.
        - products: A dictionary with reaction products as keys, and number formed per reaction as value.
        - rate: A parameter rate constant to be applied to the propensity of this reaction firing.
        - propensity_function: Can be used instead of rate in order to declare a custom propensity function
                               in string format.
        - restrict_to: Restrict reaction channels to a type or list of types within the Domain.
        """
        birth = Reaction(
            name='birth',
            reactants={},
            products={rabbits:1},
            rate=kb
        )
        
        death = Reaction(
            name='death',
            reactants={rabbits:1},
            products={},
            rate=kd
        )
        
        # Add the Reactions to the Model.
        self.add_reaction([birth, death])
        
        # Use NumPy to set the timespan of the Model.
        self.timespan(numpy.arange(0, 10, 1), timestep_size=1)

### Instantiate your Model.

In [ ]:
model = SpatialBirthDeath()

## Running Simulations

Run a spatial stochastic simulation on the Model and store the results in the 'results' variable.

In [ ]:
result = model.run()

In [ ]:

import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
result.plot_property('type')

In [ ]:
result.plot_species('rabbits', animated=True)